# Домашнее задание №2

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('data/retail_train.csv')
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [3]:
popularity = data.groupby('item_id')['sales_value'].sum().reset_index()
popularity.describe()

,item_id,sales_value
count,8.905100e+04,89051.000000
mean,5.115772e+06,83.458481
std,5.178973e+06,1628.715079
min,2.567100e+04,0.000000
25%,9.665830e+05,3.500000
50%,1.448516e+06,10.780000
75%,9.553042e+06,46.105000
max,1.802456e+07,467993.620000


### Train-test split

In [4]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [5]:
data_train.shape[0], data_test.shape[0]

(2278490, 118314)

# 1. Бейзлайны

In [6]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [7]:
test_users = result.shape[0]
new_test_users = len(set(data_test['user_id']) - set(data_train['user_id']))

print('В тестовом дата сете {} юзеров'.format(test_users))
print('В тестовом дата сете {} новых юзеров'.format(new_test_users))

В тестовом дата сете 2042 юзеров
В тестовом дата сете 0 новых юзеров


### 1.1 Random recommendation

In [8]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [9]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

result.head(2)

Wall time: 6.34 s


,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[15452774, 13213511, 1041121, 1006277, 13007479]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1008768, 1018109, 1096453, 382352, 1109615]"


### 1.2 Popularity-based recommendation

In [10]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [11]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

Wall time: 214 ms


,user_id,actual,random_recommendation,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[15452774, 13213511, 1041121, 1006277, 13007479]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1008768, 1018109, 1096453, 382352, 1109615]","[6534178, 6533889, 1029743, 6534166, 1082185]"


### 1.3 Weighted random recommender

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [12]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    recs = np.random.choice(items_weights['item_id'], 5, p=items_weights['weight'])
    
    return recs.tolist()
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    
    return recs.tolist()

In [13]:
%%time

items_weights = data_train.groupby('item_id')['sales_value'].sum().reset_index()
sum_sales_value = sum(items_weights['sales_value'])
items_weights['weight'] = items_weights['sales_value'].apply(lambda x: x / sum_sales_value)
items_weights = items_weights.drop('sales_value', 1)

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))

result.head(2)

Wall time: 2.67 s


,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[15452774, 13213511, 1041121, 1006277, 13007479]","[6534178, 6533889, 1029743, 6534166, 1082185]","[15830610, 910673, 8064581, 6544236, 1066620]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1008768, 1018109, 1096453, 382352, 1109615]","[6534178, 6533889, 1029743, 6534166, 1082185]","[863447, 918846, 1018780, 879008, 6534166]"


## Задание 2. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая случаный на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [14]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [15]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

### Random recommendation top 5000

In [16]:
%%time

items = top_5000

result['random_recommendation_top_5000'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

result.head(2)

Wall time: 1.35 s


,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation,random_recommendation_top_5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[15452774, 13213511, 1041121, 1006277, 13007479]","[6534178, 6533889, 1029743, 6534166, 1082185]","[15830610, 910673, 8064581, 6544236, 1066620]","[882190, 1020404, 6904925, 854496, 6396581]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1008768, 1018109, 1096453, 382352, 1109615]","[6534178, 6533889, 1029743, 6534166, 1082185]","[863447, 918846, 1018780, 879008, 6534166]","[916050, 9884698, 9526412, 825618, 7442008]"


### Weighted random recommender top 5000

In [17]:
popularity = data_train.groupby('item_id')[['quantity', 'sales_value']].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold,sales_value
0,25671,6,20.94
1,26081,1,0.99
2,26093,1,1.59
3,26190,1,1.54
4,26355,2,1.98


In [18]:
items_weights_top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000)
items_weights_top_5000

,item_id,n_sold,sales_value
55470,6534178,190227964,447799.94
55430,6533889,15978434,40483.34
55465,6534166,12439291,30170.77
55576,6544236,2501949,6584.34
43620,1404121,1562004,3705.77
...,...,...,...
51914,5574045,120,374.96
10610,864048,119,672.31
7777,838842,119,266.12
26071,1004001,119,295.43


In [19]:
%%time


sum_sales_value = sum(items_weights_top_5000['sales_value'])
items_weights_top_5000['weight'] = items_weights_top_5000['sales_value'].apply(lambda x: x / sum_sales_value)
items_weights_top_5000 = items_weights_top_5000.drop('sales_value', 1)
items_weights_top_5000 = items_weights_top_5000.drop('n_sold', 1)
items_weights_top_5000

Wall time: 13 ms


,item_id,weight
55470,6534178,0.110442
55430,6533889,0.009984
55465,6534166,0.007441
55576,6544236,0.001624
43620,1404121,0.000914
...,...,...
51914,5574045,0.000092
10610,864048,0.000166
7777,838842,0.000066
26071,1004001,0.000073


In [20]:
result['weighted_random_recommendation_top_5000'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights_top_5000, n=5))

result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation,random_recommendation_top_5000,weighted_random_recommendation_top_5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[15452774, 13213511, 1041121, 1006277, 13007479]","[6534178, 6533889, 1029743, 6534166, 1082185]","[15830610, 910673, 8064581, 6544236, 1066620]","[882190, 1020404, 6904925, 854496, 6396581]","[981760, 1139782, 1129671, 874972, 1029743]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1008768, 1018109, 1096453, 382352, 1109615]","[6534178, 6533889, 1029743, 6534166, 1082185]","[863447, 918846, 1018780, 879008, 6534166]","[916050, 9884698, 9526412, 825618, 7442008]","[1016800, 1071302, 999270, 987237, 1108094]"


In [21]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[ ~ data_train['item_id'].isin(top_5000), 'item_id'] = 6666
data_train.head(100)

C:\Users\1227458\Anaconda3\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1060,26985040735,1,9553288,1,8.49,315,0.00,1251,1,0.0,0.0
96,1351,26985052379,1,903230,1,0.99,447,-0.30,1955,1,0.0,0.0
97,744,26985165432,1,6666,0,0.00,31582,0.00,1119,1,0.0,0.0
98,212,26985205886,1,822346,1,1.25,288,-0.34,1341,1,0.0,0.0


In [22]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()



In [23]:
user_item_matrix.shape

(2499, 5001)

In [24]:
user_item_matrix.sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1]) * 100

5.33770796861036

In [25]:
# создаем словари мапинга между id бизнеса к строчному id матрицы

userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

# Fit

In [34]:
%%time

model = ItemItemRecommender(K=3, num_threads=4) # K - кол-во билжайших соседей


model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)


Wall time: 1.65 s


In [35]:
recs

[(0, 78679.0), (3408, 72035.0), (2149, 2953.0), (301, 1284.0), (3587, 1278.0)]

In [36]:
[id_to_itemid[rec[0]] for rec in recs]

[6666, 1082185, 981760, 840361, 1098066]

In [37]:
%%time

result['itemitem'] = result['user_id'].apply(lambda user_id: [
                                     id_to_itemid[rec[0]]  for rec in model.recommend(userid=userid_to_id[user_id], 
                                                user_items=sparse_user_item,   # на вход user-item matrix
                                                N=5, 
                                                filter_already_liked_items=False, 
                                                filter_items=None, 
                                                recalculate_user=True)
                                                             ])

Wall time: 97.7 ms


In [38]:
result.head(5)

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation,random_recommendation_top_5000,weighted_random_recommendation_top_5000,itemitem
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[15452774, 13213511, 1041121, 1006277, 13007479]","[6534178, 6533889, 1029743, 6534166, 1082185]","[15830610, 910673, 8064581, 6544236, 1066620]","[882190, 1020404, 6904925, 854496, 6396581]","[981760, 1139782, 1129671, 874972, 1029743]","[6666, 1082185, 981760, 995242, 1029743]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1008768, 1018109, 1096453, 382352, 1109615]","[6534178, 6533889, 1029743, 6534166, 1082185]","[863447, 918846, 1018780, 879008, 6534166]","[916050, 9884698, 9526412, 825618, 7442008]","[1016800, 1071302, 999270, 987237, 1108094]","[6666, 1082185, 981760, 1098066, 6534178]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[10254376, 878028, 1423201, 9488049, 13115974]","[6534178, 6533889, 1029743, 6534166, 1082185]","[5591395, 5567486, 1012873, 9419769, 1029743]","[863529, 6463710, 891217, 907994, 6464086]","[6534178, 6534178, 6534178, 1062966, 12262978]","[6666, 1082185, 981760, 995242, 1029743]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1035800, 1100546, 1674646, 161406, 9858744]","[6534178, 6533889, 1029743, 6534166, 1082185]","[13876912, 1121213, 995242, 865456, 888543]","[9297615, 884686, 979566, 1015386, 831628]","[1020156, 6534178, 6534178, 1021164, 951590]","[6666, 1082185, 981760, 995242, 1029743]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[13115658, 1024051, 8249091, 947401, 10250078]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1066095, 9832156, 6904419, 978129, 1038217]","[8119093, 1125197, 868401, 1078011, 903674]","[1076056, 5569471, 8177622, 868764, 12810391]","[6666, 1082185, 981760, 6534178, 995242]"


### 4.5 Измерим качество по precision@5

In [39]:
result.head(5)

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation,random_recommendation_top_5000,weighted_random_recommendation_top_5000,itemitem
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[15452774, 13213511, 1041121, 1006277, 13007479]","[6534178, 6533889, 1029743, 6534166, 1082185]","[15830610, 910673, 8064581, 6544236, 1066620]","[882190, 1020404, 6904925, 854496, 6396581]","[981760, 1139782, 1129671, 874972, 1029743]","[6666, 1082185, 981760, 995242, 1029743]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1008768, 1018109, 1096453, 382352, 1109615]","[6534178, 6533889, 1029743, 6534166, 1082185]","[863447, 918846, 1018780, 879008, 6534166]","[916050, 9884698, 9526412, 825618, 7442008]","[1016800, 1071302, 999270, 987237, 1108094]","[6666, 1082185, 981760, 1098066, 6534178]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[10254376, 878028, 1423201, 9488049, 13115974]","[6534178, 6533889, 1029743, 6534166, 1082185]","[5591395, 5567486, 1012873, 9419769, 1029743]","[863529, 6463710, 891217, 907994, 6464086]","[6534178, 6534178, 6534178, 1062966, 12262978]","[6666, 1082185, 981760, 995242, 1029743]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1035800, 1100546, 1674646, 161406, 9858744]","[6534178, 6533889, 1029743, 6534166, 1082185]","[13876912, 1121213, 995242, 865456, 888543]","[9297615, 884686, 979566, 1015386, 831628]","[1020156, 6534178, 6534178, 1021164, 951590]","[6666, 1082185, 981760, 995242, 1029743]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[13115658, 1024051, 8249091, 947401, 10250078]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1066095, 9832156, 6904419, 978129, 1038217]","[8119093, 1125197, 868401, 1078011, 903674]","[1076056, 5569471, 8177622, 868764, 12810391]","[6666, 1082185, 981760, 6534178, 995242]"


In [40]:
# Функции из 1-ого вебинара
import os, sys
    
from metrics import precision_at_k, recall_at_k

# Metrics

In [41]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0005:random_recommendation
0.1552:popular_recommendation
0.0217:weighted_random_recommendation
0.0053:random_recommendation_top_5000
0.0357:weighted_random_recommendation_top_5000
0.1861:itemitem


### Выводы:

- Взвешенные случайные рекомендации дают лучший результат, чем просто случайные.
- Случайные рекомендации и взвешенные случайные рекомендации топ-5000 улучшают результат.
- При уменьшении числа соседей(k) растёт качество рекомендаций.